In [ ]:
import networkx as nx
import random
import xml.etree.cElementTree as et
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from networkx.algorithms import bipartite
import json
import random
from sklearn import metrics
from collections import deque
import pandas as pd

In [ ]:
tree=et.parse('./davis.xml')
root=tree.getroot()

Agent = []
Event = []
Edges = []



# Extract agent & event from XML file

In [ ]:
for nodes in root.iter('nodeclass'):
    if(nodes.get('id') == 'agent'):
        for x in nodes:
            Agent.append(x.get('id'))
    if(nodes.get('id') == 'event'):
        for x in nodes:
            Event.append(x.get('id'))

# Extract edge pairs from XML file


In [ ]:
for networks in root.iter('networks'):
    for network in networks: 
        for links in network:
            source = links.get('source')
            target = links.get('target')
            value = links.get('value')
            if(value == "1.0000"):
                Edges.append((source,target))

In [ ]:
is_swapped = False
if len(Event) < len(Agent):
  temp = Agent.copy()
  Agent = Event.copy()
  Event = temp.copy()
  is_swapped = True

original_agent = Agent.copy()
original_event = Event.copy()
AGENT_MAP = {}
EVENT_MAP = {}
EDGES_MAP = {}
offset = len(Agent)

for idx, x in enumerate(Agent):
  AGENT_MAP[x] = idx
  Agent[idx] = idx
print(AGENT_MAP)

for idx, x in enumerate(Event):
  EVENT_MAP[x] = idx + offset
  Event[idx] = idx + offset
print(EVENT_MAP)

for idx, x in enumerate(Edges):
  if is_swapped:
    Edges[idx] = (EVENT_MAP[x[0]], AGENT_MAP[x[1]])
  else:
    Edges[idx] = (AGENT_MAP[x[0]], EVENT_MAP[x[1]])

print(Agent)
print(Event)
print(Edges)
print(len(Edges))

In [ ]:
GRAPH = nx.Graph()

GRAPH.add_nodes_from(Agent, bipartite=0)
GRAPH.add_nodes_from(Event, bipartite=1)


GRAPH.add_edges_from(Edges)
bipartite.is_bipartite(GRAPH)
nx.draw_networkx(GRAPH, pos = nx.drawing.layout.bipartite_layout(GRAPH, Agent), width = 2)

# get adjacency matrix
adj = nx.adjacency_matrix(GRAPH)
adj = adj.toarray()
# for x in adj:
#     for v in x:
#         print(v, end=" ")
#     print("\n")

U_NODES = Agent
V_NODES = Event

# plot adjacency matrix
plt.title('Adjacency Matrix')
plt.imshow(adj, cmap='Greys')
plt.show()

# plot graph visualisation
nx.draw(GRAPH, with_labels=True)

In [ ]:
# create random graph
# SMALL_GRAPH = nx.bipartite.gnmk_random_graph(6, 5, 13, seed=122)
# GRAPH = SMALL_GRAPH

# get layout
U_NODES = nx.bipartite.sets(GRAPH)[0]
V_NODES = nx.bipartite.sets(GRAPH)[1]
print(U_NODES)
print(V_NODES)
pos = nx.bipartite_layout(GRAPH, U_NODES)

# get adjacency matrix
adj = nx.adjacency_matrix(GRAPH)
adj = adj.toarray()
print(GRAPH)
# plot adjacency matrix
plt.title('Adjacency Matrix')
plt.imshow(adj, cmap='Greys')
plt.show()

# plot graph visualisation
nx.draw(GRAPH, pos, with_labels=True)

### ALGORITHM: -

In [ ]:
def PLP_ALGO1(GRAPH):
    global Agent
    global Event
    # Construct the set of all patterns
    U_PROJ_PAIRS = []

    for B in U_NODES:
        temp = [n for n in GRAPH.neighbors(B)]
        for x in temp:
            temp1 = [n for n in GRAPH.neighbors(x)]
            for C in temp1:
                if B != C and (C, B) not in U_PROJ_PAIRS and (B, C) not in U_PROJ_PAIRS:
                    U_PROJ_PAIRS.append((B, C))

    U_PROJ_PAIRS = set(U_PROJ_PAIRS)
        

    # Calculate the weight of each pattern
    weight = {}
    for pair in U_PROJ_PAIRS:
        A = pair[0]
        B = pair[1]
        
        deg_A = GRAPH.degree(A)
        deg_B = GRAPH.degree(B)
        
        neighbour_A = []
        neighbour_B = []
        for i in GRAPH.neighbors(A):
            neighbour_A.append(i)
        for i in GRAPH.neighbors(B):
            neighbour_B.append(i)

        neightbour_intersect = list(set(neighbour_A).intersection(neighbour_B))
        sum = 0
        for intersect in neightbour_intersect:
            sum += 1/(GRAPH.degree(intersect)+1)

        weight[pair] = sum * (2/(deg_A+deg_B))

    # print(weight, sep="\n")
    # print(len(weight))


    # U PROJECTED GRAPH

    U_PROJ_GRAPH = nx.projected_graph(GRAPH, U_NODES)

    # get adjacency matrix
    adj = nx.adjacency_matrix(U_PROJ_GRAPH)
    adj = adj.toarray()

    # plot adjacency matrix
    plt.title('Projected Graph Adjacency Matrix')
    plt.imshow(adj, cmap='Greys')
    plt.show()

    # # plot graph visualisation
    # plt.figure(figsize=(10,10)) 
    # plt.title("Projected Graph")
    # nx.draw(U_PROJ_GRAPH, with_labels=True)
    # plt.show()

    # Calculate the connectivity of CNPs
    U_PROJ_GRAPH_NODES = U_PROJ_GRAPH.nodes()
    mat = {}
    linked_mat = {}

    for node in U_PROJ_GRAPH_NODES:
        for nbr in U_PROJ_GRAPH.neighbors(node):
            for nbrNode in GRAPH.neighbors(nbr):
                bipartiteNodeNeighbors = []

                for node_neighbour in GRAPH.neighbors(node):
                    bipartiteNodeNeighbors.append(node_neighbour)

                CNP = (node, nbrNode)
                if nbrNode not in bipartiteNodeNeighbors:

                    if mat.get(CNP) is None:
                        mat[CNP] = 0

                    if weight.get((node, nbr)) is None:
                        weight[(node, nbr)] = 0

                    mat[CNP] = mat.get(CNP) + weight.get((node, nbr))
                else:
                    if linked_mat.get(CNP) is None:
                        linked_mat[CNP] = 0

                    if weight.get((node, nbr)) is None:
                        weight[(node, nbr)] = 0

                    linked_mat[CNP] = linked_mat.get(CNP) + weight.get((node, nbr))



    keys = list(mat.keys())
    values = list(mat.values())
    sorted_value_index = np.argsort(values)
    sorted_mat = {keys[i]: values[i] for i in sorted_value_index}

    FINAL_RESULT = []
    RESULT_IMP = []
    sum = 0

    for pr in sorted_mat:
        temp = {}
        temp[(original_agent[pr[0]], original_event[pr[1] - offset])] = sorted_mat[pr]
        RESULT_IMP.append((pr[0], pr[1], sorted_mat[pr]))
        FINAL_RESULT.append(temp)
        sum += sorted_mat[pr]

    FINAL_RESULT.reverse()
    RESULT_IMP.reverse()

    if is_swapped:
        temp = Agent.copy()
        Agent = Event.copy()
        Event = temp.copy()

    
    # print("total Predictions: ", FINAL_RESULT)
    # print("Top 10 predictions: ", FINAL_RESULT[:10])
    return [RESULT_IMP, FINAL_RESULT, mat, linked_mat]

In [ ]:
def PLP_ALGO2(GRAPH):
    global Agent
    global Event
    # Construct the set of all patterns
    U_PROJ_PAIRS = []
    adj = nx.adjacency_matrix(GRAPH)
    adj = adj.toarray()

    for B in U_NODES:
        temp = [n for n in GRAPH.neighbors(B)]
        for x in temp:
            temp1 = [n for n in GRAPH.neighbors(x)]
            for C in temp1:
                if B != C and (C, B) not in U_PROJ_PAIRS and (B, C) not in U_PROJ_PAIRS:
                    U_PROJ_PAIRS.append((B, C))

    U_PROJ_PAIRS = set(U_PROJ_PAIRS)


    # adjList :)
    adjList = [[] for i in range(len(U_NODES) + len(V_NODES) + 300)]

    for node in U_NODES:
        for nbr in adj[node]:
            if not nbr:
                continue
            adjList[node].append(nbr)

    for node in V_NODES:
        for nbr in adj[node]:
            if not nbr:
                continue
            adjList[node].append(nbr)


    def dfs(depth, node, V, maxDepth):
        if V[node]:
            return 0
        qu = deque()
        qu.append(node)
        V[node] = True
        res = 1
        while not qu and depth <= maxDepth:
            sz = len(qu)

            for i in range(sz):
                front = qu.popleft()
                V[front] = 1
                for nbr in adjList[front]:
                    if V[nbr]:
                        continue
                    if depth%2 != 0:
                        res += 1
                    qu.append(nbr)

            depth += 1

        return res
                

    # Calculate the weight of each pattern
    weight = {}
    for pair in U_PROJ_PAIRS:
        A = pair[0]
        B = pair[1]
        
        deg_A = GRAPH.degree(A)
        deg_B = GRAPH.degree(B)
        
        neighbour_A = []
        neighbour_B = []
        for i in GRAPH.neighbors(A):
            neighbour_A.append(i)
        for i in GRAPH.neighbors(B):
            neighbour_B.append(i)

        neightbour_intersect = list(set(neighbour_A).intersection(neighbour_B))
        sum = 0
        maxi = 0
        V = [False for i in range(len(U_NODES) + len(V_NODES) + 300)]
        for intersect in neightbour_intersect:
            V[A] = True
            V[B] = True
            maxDepth = 30
            denom = dfs(0, intersect, V, maxDepth) + 2
            sum = sum + 1/denom
            maxi = max(maxi, sum)
        sum = sum%(maxi+1)
        # print("sum: ", sum, sep="\n")
        weight[pair] = maxi * (2/(deg_A+deg_B))

    # print(weight, sep="\n")
    # print(len(weight))


    # U PROJECTED GRAPH

    U_PROJ_GRAPH = nx.projected_graph(GRAPH, U_NODES)

    # get adjacency matrix
    adj = nx.adjacency_matrix(U_PROJ_GRAPH)
    adj = adj.toarray()
    
    # # plot adjacency matrix
    # plt.title(' Adjacency Matrix')
    # plt.imshow(adj, cmap='Greys')
    # plt.show()

    # # plot graph visualisation
    # plt.figure(figsize=(10,10)) 
    # plt.title("Projected Graph")
    # nx.draw(U_PROJ_GRAPH, with_labels=True)
    # plt.show()


    # Calculate the connectivity of CNPs
    U_PROJ_GRAPH_NODES = U_PROJ_GRAPH.nodes()
    mat = {}
    linked_mat = {}

    for node in U_PROJ_GRAPH_NODES:
        for nbr in U_PROJ_GRAPH.neighbors(node):
            for nbrNode in GRAPH.neighbors(nbr):
                bipartiteNodeNeighbors = []

                for node_neighbour in GRAPH.neighbors(node):
                    bipartiteNodeNeighbors.append(node_neighbour)

                CNP = (node, nbrNode)
                if nbrNode not in bipartiteNodeNeighbors:

                    if mat.get(CNP) is None:
                        mat[CNP] = 0

                    if weight.get((node, nbr)) is None:
                        weight[(node, nbr)] = 0

                    mat[CNP] = mat.get(CNP) + weight.get((node, nbr))
                else:
                    if linked_mat.get(CNP) is None:
                        linked_mat[CNP] = 0

                    if weight.get((node, nbr)) is None:
                        weight[(node, nbr)] = 0

                    linked_mat[CNP] = linked_mat.get(CNP) + weight.get((node, nbr))



    keys = list(mat.keys())
    values = list(mat.values())
    sorted_value_index = np.argsort(values)
    sorted_mat = {keys[i]: values[i] for i in sorted_value_index}

    FINAL_RESULT = []
    RESULT_IMP = []
    sum = 0

    for pr in sorted_mat:
        temp = {}
        temp[(original_agent[pr[0]], original_event[pr[1] - offset])] = sorted_mat[pr]
        RESULT_IMP.append((pr[0], pr[1], sorted_mat[pr]))
        FINAL_RESULT.append(temp)
        sum += sorted_mat[pr]

    FINAL_RESULT.reverse()
    RESULT_IMP.reverse()

    if is_swapped:
        temp = Agent.copy()
        Agent = Event.copy()
        Event = temp.copy()

    
    # print("total Predictions: ", FINAL_RESULT)
    # print("Top 10 predictions: ", FINAL_RESULT[:10])
    return [RESULT_IMP, FINAL_RESULT, mat, linked_mat]

In [ ]:
def evaluate_link_prediction(pred, train_G, test_G, k=10):
    # Create arrays of true labels (y_true) and predicted scores (y_score)
    y_true = [test_G.has_edge(p[0], p[1]) for p in pred]
    y_score = [p[2] for p in pred]

    # Compute AUC-ROC score
    auc_roc = metrics.roc_auc_score(y_true, y_score) 

    # Compute AUC-PRC score
    precision, recall, thresholds = metrics.precision_recall_curve(y_true, y_score)
    auc_prc = metrics.auc(recall, precision)

    # Compute precision at k
    y_pred = [1 if p[2] > thresholds[k] else 0 for p in pred]
    precision_at_k = metrics.precision_score(y_true, y_pred)

    # Compute F1 score
    f1_score = metrics.f1_score(y_true, y_pred)

    # Print evaluation metrics
    print("AUC-ROC score: {:.4f}".format(auc_roc))
    print("AUC-PRC score: {:.4f}".format(auc_prc))
    print("Precision at k = {}: {:.4f}".format(k, precision_at_k))
    print("F1 score: {:.4f}".format(f1_score))

    # Plot auc roc graph
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.set_xlabel('Recall')
    ax1.set_ylabel('Precision')
    ax1.plot(recall, precision)
    ax1.set_title("Precision-Recall Curve")
    # plt.title('Precision-Recall Curve')

    # Compute AUC-ROC score
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score) 
    ax2.plot(fpr, tpr)
    ax2.set_title("ROC Curve, AUC = {:.2f}".format(auc_roc))
    plt.show()

    return auc_roc, auc_prc, precision_at_k, f1_score

In [ ]:
def RUN(data1, data2, idx):
    # Remove 30% of the edges
    proportion_edges = 0.3
    # this is our test set
    edge_subset = random.sample(GRAPH.edges(), int(proportion_edges * GRAPH.number_of_edges()))
    # Create a copy of the graph and remove the edges
    G_train = GRAPH.copy()
    G_train.remove_edges_from(edge_subset)

    # adjacency matrix
    A_train = nx.adjacency_matrix(G_train)
    A_train = A_train.toarray()

    plt.figure(figsize=(10,10)) 
    plt.title("Train Graph")
    nx.draw(G_train, pos, with_labels=True)
    plt.show()

    """
    TEST
    """
    G_test = nx.Graph()
    G_test.add_edges_from(edge_subset)

    plt.figure(figsize=(10,10)) 
    plt.title("Test Graph")
    nx.draw(G_test, pos, with_labels=True)
    plt.show()


    G_pred = nx.Graph()
    [RESULT1, FINAL_RESULT1, mat, linked_mat] = PLP_ALGO1(G_train)
    [RESULT2, FINAL_RESULT2, mat, linked_mat] = PLP_ALGO2(G_train)
    print(FINAL_RESULT1 == FINAL_RESULT2, RESULT1 == RESULT2)

    G_pred.add_weighted_edges_from(RESULT1)
    G_pred.add_weighted_edges_from(RESULT2)

    print("for algo1: -", sep="\n")
    [auc_roc1, auc_prc1, precision_at_k1, f1_score1] = evaluate_link_prediction(RESULT1, G_train, G_test, 3)
    print("for algo2: -", sep="\n")
    [auc_roc2, auc_prc2, precision_at_k2, f1_score2] = evaluate_link_prediction(RESULT2, G_train, G_test, 3)
    
    data1["auc_roc"].append(auc_roc1 * 100)
    data1["auc_prc"].append(auc_prc1 * 100)
    
    data2["auc_roc"].append(auc_roc2 * 100)
    data2["auc_prc"].append(auc_prc2 * 100)
    
    
    # TOP_N_PREDICTIONS1 = top_n_predictions(5, FINAL_RESULT1)
    # TOP_N_PREDICTIONS2 = top_n_predictions(5, FINAL_RESULT2)

    # file_name1 = "results_predictions_algo1/result{}.json".format(idx)
    # f = open(file_name1, "w")
    
    # # Serializing json  
    # TOP_N_PREDICTIONS1 = json.dumps(TOP_N_PREDICTIONS1, indent = 4) 
    # f.write(TOP_N_PREDICTIONS1)

    # file_name2 = "results_predictions_algo2/result{}.json".format(idx)
    # f = open(file_name2, "w")
    
    # # Serializing json  
    # TOP_N_PREDICTIONS2 = json.dumps(TOP_N_PREDICTIONS2, indent = 4) 
    # f.write(TOP_N_PREDICTIONS2)



    # PREDICTIONS_ABOVE_K1 = top_predictions_above_threshold(0.6, FINAL_RESULT1)
    # PREDICTIONS_ABOVE_K2 = top_predictions_above_threshold(0.6, FINAL_RESULT2)

    # file_name1 = "results_predictions_k_algo1/result{}.json".format(idx)
    # f = open(file_name1, "w")
    
    # # Serializing json  
    # PREDICTIONS_ABOVE_K1 = json.dumps(PREDICTIONS_ABOVE_K1, indent = 4) 
    # f.write(PREDICTIONS_ABOVE_K1)

    # file_name2 = "results_predictions_k_algo2/result{}.json".format(idx)
    # f = open(file_name2, "w")
    
    # # Serializing json  
    # PREDICTIONS_ABOVE_K2 = json.dumps(PREDICTIONS_ABOVE_K2, indent = 4) 
    # f.write(PREDICTIONS_ABOVE_K2)

    # # visualise adjacency matrix
    # Apred = nx.adjacency_matrix(G_pred)
    # Apred = Apred.toarray()
    # plt.imshow(Apredcmaps, cmap='Greys')
    # plt.show()


    # plt.figure(figsize=(10,10)) 
    # plt.title("Prediction Graph")
    # nx.draw(G_pred, pos, with_labels=True)
    # plt.show()

In [ ]:
# df1 = pd.DataFrame({'auc_roc': [], 'auc_prc': []})
# df2 = pd.DataFrame({'auc_roc': [], 'auc_prc': []})
data1 = {'auc_roc': [], 'auc_prc': []}
data2 = {'auc_roc': [], 'auc_prc': []}
for i in range(3):
    print(i+1, sep="\n")
    RUN(data1, data2, i+1)
    print("----------------------------------------END----------------------------------------\n")

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

In [ ]:
df1


In [ ]:
df2